In [115]:
import pandas as pd
import numpy as np
import os

In [116]:
#Sources


path_to_doc = "/Users/mahdihanifi/Documents/GitHub/AI-Agents-and-Agentic-AI-with-Pytho-and-Generative-AI/documents"

In [117]:
# %%script true --no-raise-error.     ## This line ensures the cell runs but does not raise errors in the notebook

# import matplotlib.pyplot as plt
# import numpy as np

# # This plot will not be saved in the notebook file
# x = np.linspace(0, 10, 100)
# plt.plot(x, np.sin(x))
# plt.show()


# SYSTEM PROMPT

### GOAL
You are an expert data analyst. Your goal is to help users by answering questions about the data inside their CSV or Excel files.

### TOOLS
You have access to the following tools:

1. Function: list_files()

Description: Lists all files and folders in the current directory.

2. Function: read_data(file_path: str)

Description: Reads a CSV or Excel file and returns the data as a table.

In [118]:
prompt = """"You are an expert data analyst. /
Your goal is to help users by answering questions /
about the data inside their CSV or Excel files.
"""

In [119]:
# def list_files(directory: str) -> list:
#     """
#     Lists all CSV and Excel files in the specified directory.
#     """
#     try:
#         files = os.listdir(directory)
#         # Filter for CSV and Excel files
#         data_files = [f for f in files if f.endswith('.csv') or f.endswith('.xlsx')]
#         return data_files
#     except FileNotFoundError:
#         return f"Error: Directory not found at {directory}"
#     except Exception as e:
#         return f"An error occurred: {e}"

# SYSTEM PROMPT (Final Version)

### GOAL
You are an expert data analyst. Your goal is to help users by answering questions about the data inside their CSV or Excel files. You must always use your tools to answer questions.

### TOOLS
You have access to the following tools:

1. Function: list_files()

Description: Lists all files and folders in the current directory. Use this to find the file you need.

2. Function: read_data(file_path: str)

Description: Reads a CSV or Excel file and returns the data as a table. Use this after you have found the file.

3. Function: analyze_data(data_table, command: str)

Description: Takes a data table and a Python command (using pandas) to perform a calculation (e.g., `data_table["Sale_Amount"].mean()` ). Returns the result of the calculation. Use this after you have read the data.

In [120]:
# --- IGNORE ---

# !pip install -q -U google-generativeai

In [121]:
import os
from dotenv import load_dotenv
load_dotenv()  # Automatically looks for ".env"

gemini_api_key = os.getenv('GEMINI_API_KEY')

import google.generativeai as genai
genai.configure(api_key=gemini_api_key)  # Set your actual API key here


from google.generativeai.types import Tool

In [122]:
path_to_doc = "/Users/mahdihanifi/Documents/GitHub/AI-Agents-and-Agentic-AI-with-Pytho-and-Generative-AI/documents"

In [123]:
# os.path.join(path_to_doc, "sample_sales.xlsx")

In [124]:
# --- Step 1: Define your Python functions (Our Tools) ---
# We'll create placeholder functions for now.
def list_files() -> list:
    """
    Lists all CSV and Excel files in the specified directory.
    """
    path = os.path.join(path_to_doc)
    try:
        files = os.listdir(path)
        # Filter for CSV and Excel files
        data_files = [f for f in files if f.endswith('.csv') or f.endswith('.xlsx')]
        return data_files
    except FileNotFoundError:
        return f"Error: Directory not found at {path}"
    except Exception as e:
        return f"An error occurred: {e}"

In [125]:
# list_files()

In [144]:
def read_data(file_name: str) -> pd.DataFrame:
    """
    Reads a CSV or Excel file from a given file path into a pandas DataFrame.
    Handles errors for non-existent files or unsupported formats.
    """
    file = os.path.join(path_to_doc, file_name)
    try:
        # Check the file extension to use the correct pandas function
        if file.endswith('.csv'):
            df = pd.read_csv(file)
            return df.to_dict(orient="records")
        elif file.endswith('.xlsx'):
            df = pd.read_excel(file)
            return df.to_dict(orient="records")
        else:
            # If the format isn't supported, we return an error message
            return "Error: Unsupported file format. Please use .csv or .xlsx."
    except FileNotFoundError:
        # If the file doesn't exist, we return an error message
        return f"Error: File not found at {file}"
    except Exception as e:
        # Catch any other potential errors during file reading
        return f"An error occurred: {e}"

In [145]:
# df = read_data(file_name= 'sample_sales.xlsx')

In [146]:
# df.info()

In [147]:
# df['Price'].dtype == 'float'
# df['Email'].dtype == 'object'
# df['Channel'].value_counts()

In [148]:
# def analyze_data(df: pd.DataFrame,  column: str, command = "sum" ):
#     if df[column].dtype == 'float':
#         """Performs a calculation on a data table using a pandas command."""
#         print(f"--- TOOL CALLED: analyze_data(command='{command}') ---")
#         # This would execute the pandas command.
#         if command == "sum":
#             return df[column].sum()
#         elif command == "mean":
#             return df[column].mean()
#         elif command == "max":
#             return df[column].max()
#         elif command == "min":
#             return df[column].min()
#         else:
#             return f"Error: Unsupported command '{command}'. Supported commands are sum, mean, max, min."
#     if df[column].dtype == 'object':
#         """Performs a calculation on a data table using a pandas command."""
#         print(f"--- TOOL CALLED: analyze_data(command='{command}') ---")
#         # This would execute the pandas command.
#         if command == "value_counts":
#             return df[column].value_counts()
#         else:
#             return f"Error: Unsupported command '{command}'. Supported commands are value_counts."
#     if df[column].dtype == 'datetime64[ns]':
#         """Performs a calculation on a data table using a pandas command."""
#         print(f"--- TOOL CALLED: analyze_data(command='{command}') ---")
#         # This would execute the pandas command.
#         if command == "max":
#             return df[column].max()
#         elif command == "min":
#             return df[column].min()
#         else:
#             return f"Error: Unsupported command '{command}'. Supported commands are max, min."


def analyze_data(df: pd.DataFrame, command: str):
    """
    Dynamically executes a pandas command on a DataFrame.
    Example command: "df['Sale_Amount'].mean()"
    """
    try:
        # The eval() function runs the code in the command string
        result = eval(command)
        return result
    except Exception as e:
        # If the command is invalid, return an error message
        return f"Error executing command: {e}"

In [149]:

# --- Step 2: Define the Agent's "Brain" ---
# Set up the model with our system prompt.
system_prompt = """
You are an expert data analyst. Your goal is to help users by answering questions
about the data inside their CSV or Excel files. You must always use your tools to
answer questions. First, list the files to find the correct one, then read it,
then perform the analysis.
"""

# Create the model with the system prompt and tool definitions
model = genai.GenerativeModel(
    'gemini-1.5-pro-latest',
    system_instruction=system_prompt,
    tools=Tool(function_declarations=[list_files, read_data, analyze_data]) 
)

# --- Step 3: Start a chat and ask a question ---
# The chat session keeps track of the conversation.
chat = model.start_chat(enable_automatic_function_calling=True)

# Your business question
user_question = "How many customers are purchasing in each channel, Store and eComm?"
print(f"User: {user_question}\n")

# Send the message to the model
response = chat.send_message(user_question)

# The model's response will automatically trigger the correct function call
# and the result will be sent back to the model in a loop until it can answer.

# --- Step 4: Print the final answer ---
# The `enable_automatic_function_calling` handles the back-and-forth for us.
print(f"Agent: {response.text}")

PydanticSchemaGenerationError: Unable to generate pydantic-core schema for <class 'pandas.core.frame.DataFrame'>. Set `arbitrary_types_allowed=True` in the model_config to ignore this error or implement `__get_pydantic_core_schema__` on your type to fully support it.

If you got this error by calling handler(<some type>) within `__get_pydantic_core_schema__` then you likely need to call `handler.generate_schema(<some type>)` since we do not call `__get_pydantic_core_schema__` on `<some type>` otherwise to avoid infinite recursion.

For further information visit https://errors.pydantic.dev/2.11/u/schema-for-unknown-type

In [114]:
?genai.GenerativeModel
# ?Tool

Init signature:
genai.GenerativeModel(
    model_name: 'str' = 'gemini-1.5-flash-002',
    safety_settings: 'safety_types.SafetySettingOptions | None' = None,
    generation_config: 'generation_types.GenerationConfigType | None' = None,
    tools: 'content_types.FunctionLibraryType | None' = None,
    tool_config: 'content_types.ToolConfigType | None' = None,
    system_instruction: 'content_types.ContentType | None' = None,
)
Docstring:     
The `genai.GenerativeModel` class wraps default parameters for calls to
`GenerativeModel.generate_content`, `GenerativeModel.count_tokens`, and
`GenerativeModel.start_chat`.

This family of functionality is designed to support multi-turn conversations, and multimodal
requests. What media-types are supported for input and output is model-dependant.

>>> import google.generativeai as genai
>>> import PIL.Image
>>> genai.configure(api_key='YOUR_API_KEY')
>>> model = genai.GenerativeModel('models/gemini-1.5-flash')
>>> result = model.generate_content(

In [68]:
def analyze_data(df: pd.DataFrame, command: str):
    """
    Dynamically executes a pandas command on a DataFrame.
    Example command: "df['Sale_Amount'].mean()"
    """
    try:
        # The eval() function runs the code in the command string
        result = eval(command)
        return result
    except Exception as e:
        # If the command is invalid, return an error message
        return f"Error executing command: {e}"

In [71]:
analyze_data(df, "df['Price'].mean()")

1059.4429599999999

In [143]:
def read_data(file_name: str) -> dict:
    import pandas as pd
    file = os.path.join(path_to_doc, file_name)
    df = pd.read_excel(file)
    return df.to_dict(orient="records")


read_data("sample_sales.xlsx")

[{'OrderID': 'ORD1000',
  'CustomerID': 'CUST365',
  'Email': 'ctorres@yahoo.com',
  'Product': 'Keyboard',
  'Quantity': 5,
  'Price': 1878.36,
  'OrderDate': Timestamp('2025-08-13 00:00:00'),
  'Region': 'South America',
  'Channel': 'Store'},
 {'OrderID': 'ORD1001',
  'CustomerID': 'CUST313',
  'Email': 'marcusgonzalez@rubio.com',
  'Product': 'Phone',
  'Quantity': 1,
  'Price': 567.7,
  'OrderDate': Timestamp('2024-11-26 00:00:00'),
  'Region': 'Europe',
  'Channel': 'Store'},
 {'OrderID': 'ORD1002',
  'CustomerID': 'CUST290',
  'Email': 'smithhunter@roberts.net',
  'Product': 'Monitor',
  'Quantity': 8,
  'Price': 268.9,
  'OrderDate': Timestamp('2024-10-18 00:00:00'),
  'Region': 'North America',
  'Channel': 'eComm'},
 {'OrderID': 'ORD1003',
  'CustomerID': 'CUST230',
  'Email': 'walterstracy@yahoo.com',
  'Product': 'Headphones',
  'Quantity': 4,
  'Price': 807.02,
  'OrderDate': Timestamp('2025-07-13 00:00:00'),
  'Region': 'Africa',
  'Channel': 'eComm'},
 {'OrderID': 'ORD10